In [ ]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.1 MB/s 


In [ ]:
!pip install tokenizers

     |████████████████████████████████| 3.3 MB 6.8 MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 6.9 MB/s 
     |████████████████████████████████| 596 kB 68.4 MB/s 
     |████████████████████████████████| 59 kB 6.1 MB/s 
     |████████████████████████████████| 895 kB 78.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub

import pandas as pd

from transformers import AutoTokenizer,AutoConfig
import numpy as np

In [ ]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4' #Loading BERT form TFhub

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [ ]:
import io
traindf= pd.read_csv(io.BytesIO(uploaded['train.csv'])) #Loading training file
testdf = pd.read_csv(io.BytesIO(uploaded['test.csv'])) #Loading Testing File

In [ ]:
testdf.iloc[0]

id                                                  22bff3dec
context     ज्वाला गुट्टा (जन्म: 7 सितंबर 1983; वर्धा, महा...
question                  ज्वाला गुट्टा की माँ का नाम क्या है
language                                                hindi
Name: 0, dtype: object

In [ ]:
bert_layer = hub.KerasLayer(tfhub_handle_encoder,trainable=True) #Loading the BERT layer from the TFhub
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased") #Loading tokenizer from the BERT multilingual model
config = AutoConfig.from_pretrained('bert-base-multilingual-cased')

tokenizer.save_pretrained('bert-base-multilingual-cased-tokenizer')
config.save_pretrained('bert-base-multilingual-cased-tokenizer')

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4'.
INFO:absl:Downloading https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4: 650.25MB
INFO:absl:Downloaded https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4, Total size: 691.54MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4'.


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

In [ ]:
traindf.shape

(1114, 6)

In [ ]:
model_name = 'bert-chaii'
max_seq_length = 384

In [ ]:
def context_offsets_index(offsets): #This will return the slice/line to which tokens are corresponding to
    last_start = 0
    idx = 0
    for x, (ostart, oend) in enumerate(offsets):
        if ostart >= last_start:
            idx = x
            last_start = ostart
        else:
            break
    return idx+1

In [ ]:
def make_samples(question, context, start_char_idx=None, answer_text=None):

    encoding = tokenizer.encode_plus(question, context,  return_tensors='np',
                                     max_length=384, stride=128, return_overflowing_tokens=True,
                                     padding="max_length", truncation=True,
                                     return_offsets_mapping=True) #This is giving us the sequence of encoded tokens

    input_word_ids = encoding.input_ids #we are doing this cause BERT needs a certain format of data in order for it to train according to that data
    token_type_ids = encoding.token_type_ids
    attention_mask = encoding.attention_mask
    offsets = encoding.offset_mapping

    # check if end character index is in the context
    end_char_idx = start_char_idx + len(answer_text)
    if end_char_idx >= len(context): #Here we are checking if end character is in the context or not
        return

    # mark all the character indexes in context that are also in answer
    is_char_in_ans = [0] * len(context)  #Now, here we mark all the characters as 0
    for idx in range(start_char_idx, end_char_idx):
        is_char_in_ans[idx] = 1 #If a character is the part of the answer , then marking it as 1
    
    inputs = []
    for x in range(len(input_word_ids)):
        ans_token_idx = []
        context_index = context_offsets_index(offsets[x])
        # find all the tokens that are in the answers
        sample_offsets = offsets[x][context_index:]
        #print('sample_offsets=', sample_offsets.shape)
        for idx, (start, end) in enumerate(sample_offsets):
            if sum(is_char_in_ans[start:end]) > 0: #As we marked all the characters which are in the answer as 1, their sum will be greater than . So we will append them in the list
                ans_token_idx.append(idx)

        if len(ans_token_idx) == 0:
            continue

        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]
        
        print('Question=', question)
        print('answer text=', answer_text)
        print('string=', tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_word_ids[x][context_index:][start_token_idx:end_token_idx+1])))

        input = {'input_word_ids': input_word_ids[x], 'input_type_ids': token_type_ids[x],
                 'input_mask': attention_mask[x], 'start_token_idx': start_token_idx,
                 'end_token_idx': end_token_idx}

        inputs.append(input)

    return inputs

In [ ]:
def chaii_train_data(df):
    train_data_samples = []
    for idx, row in df.iterrows():
        question = row['question']
        context = row['context']
        answer_start = row['answer_start']
        answer_text = row['answer_text']
        language = row['language']

        samples = make_samples(question, context, answer_start, answer_text)

        for s in samples:
            train_data_samples.append(s)

    return train_data_samples

In [ ]:
def create_bert_inputs(samples): #Here we convert our input in the form in which BERT can read it, So BERT takes dicts as input and it needs id for everything and the answer should be in token form etc.
    dataset_dict = {
        "input_word_ids": [],
        "input_type_ids": [],
        "input_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }

    for item in samples:
        for key in dataset_dict:
            dataset_dict[key].append(item[key])

    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [dataset_dict["input_word_ids"],
         dataset_dict["input_mask"],
         dataset_dict["input_type_ids"]]

    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]

    return x, y

In [ ]:
def buildModel(): #Finally building BERT and giving it Inputs
    input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_word_ids')
    input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_mask')
    input_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_type_ids')
    
    bert_inputs = {'input_word_ids': input_word_ids, 'input_mask': input_mask, 'input_type_ids': input_type_ids}
    bert_outputs = bert_layer(bert_inputs)

    pooled_output = bert_outputs["pooled_output"]      # [batch_size, 768].
    sequence_output = bert_outputs["sequence_output"]  # [batch_size, seq_length, 768].

    start_logits = tf.keras.layers.Dense(1, name="start_logit", use_bias=False)(sequence_output)
    start_logits = tf.keras.layers.Flatten()(start_logits)
    end_logits = tf.keras.layers.Dense(1, name="end_logit", use_bias=False)(sequence_output)
    end_logits = tf.keras.layers.Flatten()(end_logits)
    start_probs = tf.keras.layers.Activation(tf.keras.activations.softmax)(start_logits)
    end_probs = tf.keras.layers.Activation(tf.keras.activations.softmax)(end_logits)
    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=[start_probs, end_probs])
    
    return model

In [ ]:
def train_model(): #Here we are giving the training data to the BERT model and training the model acc to the data
    train_samples = chaii_train_data(traindf)
    x, y = create_bert_inputs(train_samples)

    print(f"{len(train_samples)} training points created.")

    model = buildModel()

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = tf.keras.optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
    model.compile(optimizer=optimizer, loss=[loss, loss])
    model.summary()

    model.fit(x, y, epochs=20, batch_size=8)
    model.save(model_name)


In [ ]:
def test_model(): #Now we prepare the test dataset,and give it to the trained model
    model = tf.keras.models.load_model(model_name)
    for idx, row in testdf.iterrows():
        id = row['id']
        question = row['question']
        context = row['context']
        encoding = tokenizer.encode_plus(question, context, return_tensors='np',
                                            max_length=384, stride=128, return_overflowing_tokens=True,
                                            padding="max_length", truncation=True,
                                            return_offsets_mapping=True)

        input_word_ids = encoding.input_ids
        token_type_ids = encoding.token_type_ids
        attention_mask = encoding.attention_mask
        offsets = encoding.offset_mapping

        input = {'input_word_ids': input_word_ids, 'input_type_ids': token_type_ids, 'input_mask': attention_mask}

        x = [input["input_word_ids"],input["input_mask"],input["input_type_ids"]]

        pred_start, pred_end = model.predict(x)

        max = 0
        answer = "unknown"
        a=[]
        b=pd.DataFrame()
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            #print('start probability=', start.max(), ' , end probability=', end.max())
            start = np.argmax(start)
            end = np.argmax(end)
            
            context_index = context_offsets_index(offsets[idx])
            # find all the tokens that are in the answers
            sample_offsets = offsets[idx][context_index:]

            if start >= end : continue
            if end >= len(sample_offsets): continue
                
            char_start = sample_offsets[start][0]
            char_end = sample_offsets[end][1]
            answer = context[char_start:char_end]
            a.append((id,start.max(),end.max(),answer))

            #print("id=", id, ", Q=",  question, ", A=", answer)
        a=pd.DataFrame(a)
        print(a)
    a.to_csv("submission.csv", index=False)

In [ ]:
train_model()

Question= மனித உடலில் எத்தனை எலும்புகள் உள்ளன?
answer text= 206
string= 206
Question= காளிதாசன் எங்கு பிறந்தார்?
answer text= காசுமீரில்
string= காசுமீரில்
Question= பென்சிலின் கண்டுபிடித்தவர் யார்?
answer text= சர் அலெக்ஸாண்டர் ஃபிளெமிங்
string= சர் அலெக்ஸாண்டர் ஃபிளெமிங்
Question= தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பாடல் எவ்வாறு அழைக்கப்படுகிறது?
answer text= தாலாட்டு
string= தாலாட்டு
Question= பூமியின் அருகில் உள்ள விண்மீன் எது?
answer text= சூரியனும்
string= சூரியனும்
Question= திரிதடையங்களைப் பயன்படுத்திய முதல் நிறுவனம் எது?
answer text= IBM
string= IBM
Question= திரிதடையங்களைப் பயன்படுத்திய முதல் நிறுவனம் எது?
answer text= IBM
string= IBM
Question= தொலைபேசியைக் கண்டுபிடித்தவர் யார்?
answer text= அலெக்ஸாண்டர் கிரகாம் பெல்
string= அலெக்ஸாண்டர் கிரகாம் பெல்
Question= முதலாம் உலகப்போர் எப்பொழுது துவங்கியது?
answer text= 1914ம்
string= 1914ம்
Question= இந்தியாவில் மனித உரிமை ஆணையம் எப்போது நிறுவப்பட்டது?
answer text= அக்டோபர் 12, 1993
string= அக்டோபர் 12, 1993
Question= நெல்ச

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_mask (InputLayer)        [(None, 384)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 384)]        0           []                               
                                                                                                  
 input_word_ids (InputLayer)    [(None, 384)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'encoder_outputs':  177853441   ['input_mask[0][0]',             
                                 [(None, 384, 768),               'input_type_ids[0][0]',     

KeyboardInterrupt: ignored

In [ ]:
test_model()